In [36]:
import sys
import os
import argparse, json, math
from pathlib import Path
import pandas as pd
import numpy as np
import tensorflow as tf, os
from tqdm import tqdm
tf.get_logger().setLevel('ERROR') # only show error messages


from recommenders.models.sasrec.model import SASREC
from recommenders.models.sasrec.ssept import SSEPT
from recommenders.models.sasrec.sampler import WarpSampler
from recommenders.models.sasrec.util import SASRecDataSet
from recommenders.utils.notebook_utils import store_metadata
from recommenders.utils.timer import Timer


print(f"System version: {sys.version}")
print(f"Tensorflow version: {tf.__version__}")

System version: 3.9.18 | packaged by conda-forge | (main, Dec 23 2023, 16:33:10) 
[GCC 12.3.0]
Tensorflow version: 2.14.0


In [37]:
gpus = tf.config.list_physical_devices('GPU')
assert gpus, "❌ No GPU visible"
print("✅ GPU device:", gpus[0].name)       # should contain “A100”


# faster allocator (TensorFlow >= 2.10)
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

✅ GPU device: /physical_device:GPU:0


In [38]:
num_epochs = 5
batch_size = 512
seed = 100  # Set None for non-deterministic result

# data_dir = os.path.join("tests", "recsys_data", "RecSys", "SASRec-tf2", "data")
DATA_DIR = Path("../../../data/processed")
SEQ_FILE = DATA_DIR / "seq_ready.txt"    # final tab–separated file
CKPT_DIR = CKPT_DIR = Path("data/ckpt")

lr = 0.001             # learning rate
maxlen = 50            # maximum sequence length for each user
num_blocks = 2         # number of transformer blocks
hidden_units = 100     # number of units in the attention calculation
num_heads = 1          # number of attention heads
dropout_rate = 0.1     # dropout rate
l2_emb = 0.0           # L2 regularization coefficient
num_neg_test = 100     # number of negative examples per positive example
model_name = 'ssept'  # 'sasrec' or 'ssept'

In [39]:
dataset = SASRecDataSet(filename=str(SEQ_FILE), col_sep="\t")
dataset.split()                                       # train / valid / test dicts


num_steps = int(len(dataset.user_train) / batch_size)
cc = 0.0
for u in dataset.user_train:
    cc += len(dataset.user_train[u])
print('%g Users and %g items' % (dataset.usernum, dataset.itemnum))
print('average sequence length: %.2f' % (cc / len(dataset.user_train)))

56208 Users and 4697 items
average sequence length: 8.53


In [40]:
sampler = WarpSampler(dataset.user_train,
                      dataset.usernum, dataset.itemnum,
                      batch_size=batch_size,
                      maxlen=maxlen, n_workers=3)

In [41]:

if model_name == 'sasrec':
    model = SASREC(item_num=dataset.itemnum,
                   seq_max_len=maxlen,
                   num_blocks=num_blocks,
                   embedding_dim=hidden_units,
                   attention_dim=hidden_units,
                   attention_num_heads=num_heads,
                   dropout_rate=dropout_rate,
                   conv_dims = [100, 100],
                   l2_reg=l2_emb,
                   num_neg_test=num_neg_test
    )
elif model_name == "ssept":
    model = SSEPT(item_num=dataset.itemnum,
                  user_num=dataset.usernum,
                  seq_max_len=maxlen,
                  num_blocks=num_blocks,
                  embedding_dim=hidden_units,
                  user_embedding_dim=10,
                  item_embedding_dim=hidden_units,
                  attention_dim=hidden_units,
                  attention_num_heads=num_heads,
                  dropout_rate=dropout_rate,
                  conv_dims = [110, 110],
                  l2_reg=l2_emb,
                  num_neg_test=num_neg_test
    )
else:
    print(f"Model-{model_name} not found")

In [42]:

with Timer() as train_time:
    t_test = model.train(dataset, sampler,
                         num_epochs=num_epochs,
                         batch_size=batch_size,
                         lr=lr, val_epoch=6,
                         model_path = str(CKPT_DIR))

print('Time cost for training is {0:.2f} mins'.format(train_time.interval/60.0))


epoch: 5, test (NDCG@10: 0.5999530752781915, HR@10: 0.7717775634844102)
Time cost for training is 2.48 mins


In [43]:
store_metadata("ndcg@10", t_test[0])
store_metadata("Hit@10", t_test[1])

In [44]:
# after training:
ckpt = tf.train.Checkpoint(model=model)
ckpt.write(str(CKPT_DIR / "sasrec.ckpt"))


'data/ckpt/sasrec.ckpt'

In [45]:
import json

config = {
    "item_num": dataset.itemnum,
    "seq_max_len": maxlen,
    "num_blocks": num_blocks,
    "embedding_dim": hidden_units,
    "attention_dim": hidden_units,
    "attention_num_heads": num_heads,
    "dropout_rate": dropout_rate,
    "conv_dims": [100, 100],
    "l2_reg": l2_emb,
    "num_neg_test": num_neg_test
}
with open(CKPT_DIR / "sasrec_config.json", "w") as f:
    json.dump(config, f)